In [4]:
def fizz_buzz_encode(i):
    if i % 15 == 0: return 3
    elif i % 5 == 0: return 2
    elif i % 3 == 0: return 1
    else: return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), 'fizz', 'buzz', 'fizzbuzz'][prediction]

def helper(i):
    return fizz_buzz_decode(i, fizz_buzz_encode(i))

for i in range(1, 16):
    print(helper(i))
    

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz


### 定义训练数据

In [108]:
import numpy as np
import torch

NUM_DIGITS = 10

def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

# print(binary_encode(16, 10))

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])
trY[:10]

tensor([0, 1, 0, 0, 3, 0, 0, 1, 0, 2])

### 定义模型

In [101]:
NUM_HIDDEN = 100

model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

BATCH_SIZE = 128
for epoch in range(3000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start: end]
        batchY = trY[start: end]
        
        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    loss = loss_fn(model(trX), trY).item()
    if epoch % 1000 == 999:
        print(epoch, loss)

999 0.0002462750126142055
1999 2.6582381451589754e-06
2999 3.215938448875022e-08


In [102]:
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
with torch.no_grad():
    testY = model(testX)
# print(testY.max(1)[1])

predictions = zip(range(1, 101), testY.max(1)[1])

result = np.array([fizz_buzz_decode(i, x) for (i, x) in predictions])

In [103]:
p_true = [fizz_buzz_encode(i) for i in range(1, 101)]

result_true = [fizz_buzz_decode(i, x) for (i, x) in zip(range(1, 101), p_true)]

count = np.sum(p_true == testY.max(1)[1].numpy())
count/len(range(1, 101))

0.97

In [104]:
print(np.sum(testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])))
index = testY.max(1)[1].numpy() != np.array([fizz_buzz_encode(i) for i in range(1,101)])

97


In [105]:
r = np.arange(1, 101)
r[index]

print(f"num:{r[index]}, res:{result[index]}")

num:[ 2 26 47], res:['fizz' 'fizz' 'fizz']
